In [159]:
import pandas as pd
from rltk.similarity.tf_idf import TF_IDF

* Divide sheet into tables (Preserve rows and columns indices )
    * Pandas indices start from zero whereas for sheet it starts from 1
* Link source annotations to tables in source DF
* Link source tables to target tables
    * Based on similarity score. E.g. Number of exact same cells between 2 tables
* Set target annotation bounds based by the linking
    * Consider irregularities in the table
* Generate annotations

In [160]:
def get_blocks(series):
    last_index = len(series) - 1
    blocks = []
    block = {}
    for index, is_empty in series.iteritems():
        if not block and not is_empty:
            block['start'] = index

        if block and not is_empty:
            block['end'] = index

        if block and (is_empty or index == last_index):
            blocks.append(block)
            block = {}
    return blocks

In [161]:
def extract_tables(dataframe, prefix):
    table_count = 0
    extracted_tables = {}

    rows = dataframe.isnull().all(1)
    row_blocks = get_blocks(rows)
    for row_block in row_blocks:
        row_df = dataframe[row_block['start']:row_block['end'] + 1]
        columns = row_df.isnull().all()
        column_blocks = get_blocks(columns)
        for column_block in column_blocks:
            extracted_tables[f'{prefix}_{str(table_count)}'] = row_df.iloc[:, column_block['start']:column_block['end'] + 1]
            table_count += 1

    return extracted_tables

In [162]:
source_df = pd.read_excel('../resources/data.xlsx', sheet_name='shifted_india_wheat', engine='openpyxl', index_col=None, header=None)
target_df = pd.read_excel('../resources/data.xlsx', sheet_name='usa_wheat', engine='openpyxl', index_col=None, header=None)

source_tables = extract_tables(source_df, 'source')
target_tables = extract_tables(target_df, 'target')

tables = {**source_tables, **target_tables}

tfidf = TF_IDF()
for table_name, table in tables.items():
    data = []
    for rowIndex, row in table.iterrows():  #iterate over rows
        for columnIndex, value in row.items():
            if isinstance(value, str):
                data.append(value)

    tfidf.add_document(table_name, data)

tfidf.pre_compute()

In [163]:
for s in range(0, len(source_tables)):
    source = f'source_{s}'

    best_similarity = 0
    best_match = ''
    for t in range(0, len(target_tables)):
        target = f'target_{t}'
        similarity = tfidf.similarity(source, target)
        # print(f'{source} & {target}: {similarity}')
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = target

    print(f'Match: {source} & {best_match}: {best_similarity}')
    #
    # print('********************************************************')
    # print('********************************************************')
    # print(f'Match: {source} & {best_match}: {best_similarity}')
    # print('********************************************************')
    # print(tables[source])
    # print('********************************************************')
    # print(tables[best_match])

Match: source_0 & target_0: 0.6363636363636362
Match: source_1 & target_1: 1.0
Match: source_2 & target_2: 0.19777158774373269
Match: source_3 & target_3: 1.0
Match: source_4 & target_4: 1.0
Match: source_5 & target_5: 1.0
Match: source_6 & target_6: 1.0000000000000002
